In [2]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')


In [14]:
# parameters
customer_id = '5'
formatted_attribute = 'Mount Type'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Hair Styling Tools"""

regex_pattern=r'(?i)(cord length measures\s?\d+.?\d*.?\")|((cord length measures\s?\d+.?\d*.?ft))()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [15]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [16]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/uncurated.sql', params=params)
df

product_id external_id  \
0       817659923    69283529   
1      1066091430    69536451   
2      1071958192    62130585   
3       817662785    46783875   
4       817380562    68693510   
...           ...         ...   
24441  1155489343    18722003   
24442   817541247    63478013   
24443   817542295    63556278   
24444   817637264    69521831   
24445   817456383    40060804   

                                                             product_name  \
0                                 X-Frame 3-Shelf Bathroom Tower in White   
1                                                        WALL SHELF BROWN   
2                             Arcadia Garden Swoop Planter in Terra Cotta   
3                                Campania Beldon Planter in Antique White   
4                               Bee Happy Personalized Outdoor Flower Pot   
...                                                                   ...   
24441                         ELK Lighting 1- Light Sconce in Silver Leaf   
24442  Designs Direct Photo Upload 14-Inch x 20-Inch Pallet Wood Wall Art   
24443             Private Family Beach 22-Inch x 17.81-Inch Wood Wall Art   
24444            Rainbow Personalized Easter Basket With Drop-Down Handle   
24445  Boon Twig Grass and Lawn Countertop Drying Rack Accessory in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                        This beautiful X-Frame Bathroom Tower is a stylish and efficient storage solution for your bathroom. This sturdy tower features an X-frame design that can be paired with any décor. It is perfect for soaps, towels, and more. X-frame design 3 open shelves MDF/veneer Wipe clean Measures 11\" L x 14\" D x 29.5\" H Weighs 23.1 lb. Holds up to 40 lb. Anti-tip kit included No assembly required 1-year limited manufacturer's warranty Imported Coordinate with other items in the X-Frame Collection for a complete look (sold separately)   
1                                                                                                                                                                               This beautiful X-Frame Bathroom Wall Shelf is a stylish and efficient storage solution for your bathroom. This sturdy shelf features an X-frame design that can be paired with any décor. It is perfect for soaps, towels, and more. X-frame design MDF Wipe clean with a soft, damp cloth Measures 20\" L x 8.5\" D x 12\" H Weighs 18 lb. Holds up to 20 lb. Wall mount No assembly required 1-year limited manufacturer's warranty Imported Coordinate with other items in the X-Frame Collection for a complete look (sold separately)   
2                                                                                                                                                                          A perfect accent for your patio, the Arcadia Garden Swoop Planter will add a lovely vibe to your space. Constructed out of a unique blend of stone, plastic, and wood, the swoop pot features a modern curved silhouette for a modern flair. Arcadia Garden Swoop Planter will add a charming touch to any patio or deck Modern curved design brings an elegant flair Each pot has a unique marbled effect 33% plastic/3

In [17]:
# error

In [19]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['Adhesive Mount',
 'Ceiling Mount',
 'Concealed Mount',
 'Corner Mount',
 'Cupboard Mount',
 'Door Mount',
 'Flush Mount',
 'Frame Mount',
 'Freestanding',
 'Hanging',
 'Hinge Mount',
 'Inside Mount',
 'Leaning',
 'Magnetic Mount',
 'Outside Mount',
 'Pole Mount',
 'Post Mount',
 'Pre-Drilled Mounting Holes',
 'Recessed Mount',
 'Roof Mount',
 'Screw Fittings',
 'Semi-Flush Mount',
 'Suction Mount',
 'Tension Mount',
 'Under Cabinet Mount',
 'Wall Mount']

In [20]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

None


0                  []
1        [Wall Mount]
2                  []
3                  []
4                  []
             ...     
24441              []
24442              []
24443       [Hanging]
24444              []
24445              []
Name: long_desc, Length: 24446, dtype: object

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df=df[df['matches'].astype(str)!='[]']
df['matches'].explode().value_counts()

Induction    57
Electric      3
Gas           2
Name: matches, dtype: int64

In [9]:
# error

In [10]:
matches=df
# matches['matches']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches.head(1)

product_id external_id  \
1   817426705    69585969   

                                             product_name  \
1  Le Creuset® Toughened Nonstick Pro 2-Piece Fry Pan Set   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
1  The Le Creuset Toughened Nonstick Pro Fry Pan Set ensures all the ease of nonstick cooking on a surface that will never chip or flake. Hard-anodized, fully encapsulated aluminum core conducts heat quickly and evenly for perfect results. Delivers unsurpassed nonstick performance on a surface that will never chip or flake 2-piece set includes: 9\" fry pan 11\" fry pan Hard-anodized, fully encapsulated aluminum core conducts heat quickly and evenly Nonstick surface is now 4x stronger with a triple-reinforced PFOA-free coating to prevent flaking, peeling and rusting Textured interior coating browns and sears exceptionally well to ensure food is cooked to perfection, with or without the use of oil, but still easily wipes clean Magnetic stainless steel base disk resists corrosion Compatible with all stove types, including induction Covered cookware features a tempered glass lid to let you visibly monitor cooking progress without letting heat and moisture from escaping Stay-cool ergonomic handles allow for quick, comfortable movement   

   customer_id attributes                 buckets values      matches  
1            5       None  Frying Pans & Skillets   None  [Induction]

In [11]:
test_list=matches['matches']
test_list

1                                      [Induction]
2                                      [Induction]
3                                      [Induction]
5                             [Electric, Electric]
6                                      [Induction]
7                           [Induction, Induction]
8                                      [Induction]
9                           [Induction, Induction]
11                                     [Induction]
12                          [Induction, Induction]
13    [Induction, Induction, Induction, Induction]
17    [Induction, Induction, Induction, Induction]
18                          [Induction, Induction]
19                                     [Induction]
21                                     [Induction]
22                          [Induction, Induction]
27                                     [Induction]
28                          [Induction, Induction]
29                          [Induction, Induction]
30                          [In

In [23]:
list(set(test_list[5]))

['Electric']

In [13]:
range(len(test_list))

range(0, 37)

In [18]:
new_df=pd.DataFrame()
for i in (1,38):
    new_df.append(list(set(test_list[i])))
new_df

Empty DataFrame
Columns: []
Index: []

In [15]:
# test_list=lambda x: [list(set(test_list[i])) for i in test_list]
# test_list

In [16]:
matches['matches'].astype(str).apply(lambda x: list(set(test_list[i])) for i in range(len(test_list)))

TypeError: 'generator' object is not callable

In [ ]:
error

In [ ]:
matches[curation_col] = matches['Max'].astype(str)+' in'
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)